# Financial Analysis with Langchain and CrewAI

**_This weekend I am experimenting how two frameworks that overlap compare with each other, langchain and crewAI._**

**_For the most part this is not a fair experiment, since I'm using Cohere's cmd r for langchain and GPT 3.5 for crewAI and IMO cmd R is better at understanding and following instructions.
I also don't have much experience with crewai and this is my attempt to learn it_**

How to Become an Expert at Something (Attempting to Be an Expert)
1. Iteratively take on concrete projects and accomplish them in depth, learning "on demand" (i.e., don't learn from the bottom up).
2. Teach/Summarize everything you learn in your own words.
3. Only compare yourself to your younger self (comparing to others may discourage you).

In [1]:
import os
from typing import List

import cohere
import langchain
import langchain_core
import langchain_experimental
import pandas as pd
from langchain.agents import Tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.utilities import PythonREPL

In [2]:
####################################################################################################
#
# Uncomment if you need to install the following packages
#
####################################################################################################

# %pip install --quiet langchain langchain_experimental cohere --upgrade

In [3]:
# versions
print('cohere version:', cohere.__version__)
print('langchain version:', langchain.__version__)
print('langchain_core version:', langchain_core.__version__)
print('langchain_experimental version:', langchain_experimental.__version__)

cohere version: 5.5.8
langchain version: 0.2.6
langchain_core version: 0.2.10
langchain_experimental version: 0.0.62


### API Key 

Following a guide on cohere's website I chose to use their model CMD-R (It's free unlike others)

In [6]:
COHERE_API_KEY = os.environ["COHERE_API_KEY"] 
CHAT_URL= "https://api.cohere.ai/v1/chat"
COHERE_MODEL = 'command-r-plus'
co = cohere.Client(api_key=COHERE_API_KEY)

### Data Loading 

In [15]:
income_statement = pd.read_csv('income_statement.csv', index_col=0)
balance_sheet = pd.read_csv('balance_sheet.csv', index_col=0)

In [10]:
income_statement.head(2)

,index,RevenueFromContractWithCustomerExcludingAssessedTax,CostOfGoodsAndServicesSold,GrossProfit,ResearchAndDevelopmentExpense,SellingGeneralAndAdministrativeExpense,OperatingExpenses,OperatingIncomeLoss,NonoperatingIncomeExpense,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeTaxExpenseBenefit,NetIncomeLoss,EarningsPerShareBasic,EarningsPerShareDiluted,WeightedAverageNumberOfSharesOutstandingBasic,WeightedAverageNumberOfDilutedSharesOutstanding
0,2017-10-01-2018-09-29,265595000000,1.637560e+11,101839000000,1.423600e+10,1.670500e+10,3.094100e+10,7.089800e+10,2.005000e+09,7.290300e+10,1.337200e+10,59531000000,3.00,2.98,1.982151e+10,2.000044e+10
1,2018-09-30-2018-12-29,84310000000,NaN,32031000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19965000000,1.05,1.05,NaN,NaN


In [12]:
balance_sheet.tail(2)

,index,CashAndCashEquivalentsAtCarryingValue,MarketableSecuritiesCurrent,AccountsReceivableNetCurrent,InventoryNet,NontradeReceivablesCurrent,OtherAssetsCurrent,AssetsCurrent,MarketableSecuritiesNoncurrent,PropertyPlantAndEquipmentNet,...,LongTermDebtNoncurrent,OtherLiabilitiesNoncurrent,LiabilitiesNoncurrent,Liabilities,CommitmentsAndContingencies,CommonStocksIncludingAdditionalPaidInCapital,RetainedEarningsAccumulatedDeficit,AccumulatedOtherComprehensiveIncomeLossNetOfTax,StockholdersEquity,LiabilitiesAndStockholdersEquity
2,2019-09-28,4.884400e+10,5.171300e+10,2.292600e+10,4.106000e+09,2.287800e+10,1.235200e+10,1.628190e+11,1.053410e+11,3.737800e+10,...,9.180700e+10,5.050300e+10,1.423100e+11,2.480280e+11,0.0,4.517400e+10,4.589800e+10,-584000000.0,90488000000,3.385160e+11
3,2020-09-26,3.801600e+10,5.292700e+10,1.612000e+10,4.061000e+09,2.132500e+10,1.126400e+10,1.437130e+11,1.008870e+11,3.676600e+10,...,9.866700e+10,5.449000e+10,1.531570e+11,2.585490e+11,0.0,5.077900e+10,1.496600e+10,-406000000.0,65339000000,3.238880e+11


<a id="define_python_tool"></a>

# Define Python Tool 

Here we define the python tool using langchain's PythonREPL. We also define `functions_map` that will later be used by the Cohere Agent to correctly map function name to the actual function. Lastly, we define the tools that will be passed in the Cohere API. 


In [13]:
python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
python_tool.name = "python_interpreter"

class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
python_tool.args_schema = ToolInput

def run_python_code(code: str) -> dict:
    """
    Function to run given python code
    """
    input_code = ToolInput(code=code)
    return {'python_answer': python_tool.func(input_code.code)}

functions_map = {
    "run_python_code": run_python_code,
}

tools = [
    {
        "name": "run_python_code",
        "description": "given a python code, runs it",
        "parameter_definitions": {
            "code": {
                "description": "executable python code",
                "type": "str",
                "required": True
            }
        }
    },]

In [14]:
def cohere_agent(
    message: str,
    preamble: str,
    tools: List[dict],
    force_single_step=False,
    verbose: bool = False,
) -> str:
    """
    Function to handle multi-step tool use api.

    Args:
        message (str): The message to send to the Cohere AI model.
        preamble (str): The preamble or context for the conversation.
        tools (list of dict): List of tools to use in the conversation.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        str: The final response from the call.
    """

    counter = 1

    response = co.chat(
        model=COHERE_MODEL,
        message=message,
        preamble=preamble,
        tools=tools,
        force_single_step=force_single_step,
    )

    if verbose:
        print(f"\nrunning 0th step.")
        print(response.text)

    while response.tool_calls:
        tool_results = []

        if verbose:
            print(f"\nrunning {counter}th step.")

        for tool_call in response.tool_calls:
            output = functions_map[tool_call.name](**tool_call.parameters)
            outputs = [output]
            tool_results.append({"call": tool_call, "outputs": outputs})

            if verbose:
                print(
                    f"= running tool {tool_call.name}, with parameters: {tool_call.parameters}"
                )
                print(f"== tool results: {outputs}")

        response = co.chat(
            model=COHERE_MODEL,
            message="",
            chat_history=response.chat_history,
            preamble=preamble,
            tools=tools,
            force_single_step=force_single_step,
            tool_results=tool_results,
        )

        if verbose:
            print(response.text)

            counter += 1

    return response.text


# test
output = cohere_agent("can you use python to answer 1 + 1", None, tools, verbose=True)

Python REPL can execute arbitrary code. Use with caution.



running 0th step.
I will use Python to answer this question.

running 1th step.
= running tool run_python_code, with parameters: {'code': 'print(1 + 1)'}
== tool results: [{'python_answer': '2\n'}]
The answer is **2**.


<a id="qna_over_single_table"></a>

# QnA over Single Table 

In the example below, we show how the python tool can be used to load a dataframe and extract information from it. To do this successfully we need to:

1) pass the file name to the preamble so the model knows how to load the dataframe
2) pass a preview of the dataframe in the preamble so the model knows which columns/rows to query

We will ask the following questions given income statement data. 
1. what is the highest value of cost of goods and service?
2. what is the largest gross profit margin?
3. what is the minimum ratio of operating income loss divided by non operating income expense? 

In [15]:
question_dict ={
    'q1': ['what is the highest value of cost of goods and service?',169559000000],
    'q2': ['what is the largest gross profit margin?',0.3836194330595236],
    'q3': ['what is the minimum ratio of operating income loss divided by non operating income expense?',35.360599]
}

In [16]:
preamble = """
You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is `income_statement.csv`.
Here is a preview of the dataframe:
{head_df}
""".format(head_df=income_statement.head(3).to_markdown())

print(preamble)


You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is `income_statement.csv`.
Here is a preview of the dataframe:
|    | index                 |   RevenueFromContractWithCustomerExcludingAssessedTax |   CostOfGoodsAndServicesSold |   GrossProfit |   ResearchAndDevelopmentExpense |   SellingGeneralAndAdministrativeExpense |   OperatingExpenses |   OperatingIncomeLoss |   NonoperatingIncomeExpense |   IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest |   IncomeTaxExpenseBenefit |   NetIncomeLoss |   EarningsPerShareBasic |   EarningsPerShareDiluted |   WeightedAverageNumberOfSharesOutstandingBasic |   WeightedAverageNumberOfDilutedSharesOutstanding |
|---:|:----------------------|------------------------------------------------------:|-----------------------------:|--------------:|--------------------------------:|-----------------------------------------:|-------------

In [17]:
for qsn,val in question_dict.items():
    print(f'question:{qsn}')
    question = val[0]
    answer = val[1]
    output = cohere_agent(question, preamble, tools, verbose=True)
    print(f'GT Answer:{val[1]}')
    print('-'*50)

question:q1

running 0th step.
I will write and execute Python code to find the highest value of 'CostOfGoodsAndServicesSold' in the dataframe.

running 1th step.
= running tool run_python_code, with parameters: {'code': 'import pandas as pd\n\ndf = pd.read_csv(\'income_statement.csv\')\n\n# Find the highest value of \'CostOfGoodsAndServicesSold\'\nhighest_cost = df[\'CostOfGoodsAndServicesSold\'].max()\n\nprint(f"The highest value of \'CostOfGoodsAndServicesSold\' is {highest_cost}")'}
== tool results: [{'python_answer': "The highest value of 'CostOfGoodsAndServicesSold' is 169559000000.0\n"}]
The highest value of 'CostOfGoodsAndServicesSold' is 169559000000.0.
GT Answer:169559000000
--------------------------------------------------
question:q2

running 0th step.
I will write and execute Python code to find the largest gross profit margin.

running 1th step.
= running tool run_python_code, with parameters: {'code': "import pandas as pd\n\ndf = pd.read_csv('income_statement.csv')\n\n#

<a id="qna_over_multiple_tables"></a>

# QnA over Multiple Tables 

We now make the task for the Agent more complicated, by asking it a question that answer can be computed only by retrieving relevant information from multiple tables: 

- Q: What is the ratio of the largest stockholders equity to the smallest revenue?

As you will see below, this question can be obtained only by accessing both the balance sheet and the income statement. 



In [21]:
question_dict ={
    'q1': ['what is the ratio of the largest stockholders equity to the smallest revenue'],
}

In [22]:
# get the largest stockholders equity
x = balance_sheet['StockholdersEquity'].astype(float).max()
print(f"The largest stockholders equity value is: {x}")

# get the smallest revenue
y = income_statement['RevenueFromContractWithCustomerExcludingAssessedTax'].astype(float).min()
print(f"The smallest revenue value is: {y}")

# compute the ratio
ratio = x/y
print(f"Their ratio is: {ratio}")

The largest stockholders equity value is: 134047000000.0
The smallest revenue value is: 53809000000.0
Their ratio is: 2.4911631883142227


In [23]:
preamble = """
You are an expert who answers the user's question in complete sentences. You are working with two pandas dataframe in Python. Ensure your output is a string.

Here is a preview of the `income_statement.csv` dataframe:
{table_1}

Here is a preview of the `balance_sheet.csv` dataframe:
{table_2}
""".format(table_1=income_statement.head(3).to_markdown(),table_2=balance_sheet.head(3).to_markdown())


print(preamble)


You are an expert who answers the user's question in complete sentences. You are working with two pandas dataframe in Python. Ensure your output is a string.

Here is a preview of the `income_statement.csv` dataframe:
|    | index                 |   RevenueFromContractWithCustomerExcludingAssessedTax |   CostOfGoodsAndServicesSold |   GrossProfit |   ResearchAndDevelopmentExpense |   SellingGeneralAndAdministrativeExpense |   OperatingExpenses |   OperatingIncomeLoss |   NonoperatingIncomeExpense |   IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest |   IncomeTaxExpenseBenefit |   NetIncomeLoss |   EarningsPerShareBasic |   EarningsPerShareDiluted |   WeightedAverageNumberOfSharesOutstandingBasic |   WeightedAverageNumberOfDilutedSharesOutstanding |
|---:|:----------------------|------------------------------------------------------:|-----------------------------:|--------------:|--------------------------------:|-----------------------------

In [24]:
for qsn,val in question_dict.items():
    print(f'question:{qsn}')
    question = val[0]
    output = cohere_agent(question, preamble, tools, verbose=True)

question:q1

running 0th step.
I will use Python to find the largest and smallest values of the relevant columns in the dataframes, and then calculate the ratio of the largest stockholder's equity to the smallest revenue.

running 1th step.
= running tool run_python_code, with parameters: {'code': 'import pandas as pd\n\n# Read the data into dataframes\nincome_statement = pd.read_csv(\'income_statement.csv\')\nbalance_sheet = pd.read_csv(\'balance_sheet.csv\')\n\n# Find the smallest revenue\nsmallest_revenue = income_statement[\'RevenueFromContractWithCustomerExcludingAssessedTax\'].min()\n\n# Find the largest stockholder\'s equity\nlargest_stockholders_equity = balance_sheet[\'StockholdersEquity\'].max()\n\n# Calculate the ratio\nratio = largest_stockholders_equity / smallest_revenue\nprint(f"The ratio of the largest stockholder\'s equity to the smallest revenue is {ratio:.2f}")'}
== tool results: [{'python_answer': "The ratio of the largest stockholder's equity to the smallest revenu

<a id="error_resilience"></a>

# Error Resilience

In the previous example over single table, the model successfully answered your questions. However, the model may not always have access to the preview of the data. You will see that when we remove the preview from the preamble, the model is run into an error and not produce the answer. We will solve this problem with two different ways: 

1. Asking the model to keep trying until it fixes the issue. 
2. Giving the model another tool to view the data and telling it to preview the data before writing code. 

You will see that the second method is able to come to the answer with fewer steps. 


In [25]:
preamble = """
You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is `income_statement.csv`.
"""

question1 = "what is the highest value of cost of goods and service?"

output = cohere_agent(question1, preamble, tools, verbose=True)


running 0th step.
I will use Python to find the highest value of 'cost of goods and service'.

running 1th step.
= running tool run_python_code, with parameters: {'code': 'import pandas as pd\n\ndf = pd.read_csv(\'income_statement.csv\')\n\n# Find the highest value of \'cost of goods and service\'\nhighest_value = df[\'cost of goods and service\'].max()\nprint(f"The highest value of \'cost of goods and service\' is {highest_value}")'}
== tool results: [{'python_answer': "KeyError('cost of goods and service')"}]
Sorry, there is no column named 'cost of goods and service' in the dataframe.


As you see above, the model failed to execute because it assumed certain column names but they turned out to be wrong. One simple fix is to tell the model to continue to solve the problem in the face of error. 

In [26]:
preamble = """
You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is `income_statement.csv`.
If you run into error, keep trying until you fix it. You may need to view the data to understand the error.
"""

question1 = "what is the highest value of cost of goods and service?"

output = cohere_agent(question1, preamble, tools, verbose=True)


running 0th step.
I will use Python to find the highest value of cost of goods and service.

running 1th step.
= running tool run_python_code, with parameters: {'code': 'import pandas as pd\n\ndf = pd.read_csv(\'income_statement.csv\')\n\n# Find the highest value of \'Cost of Goods and Services\'\nhighest_cost = df[\'Cost of Goods and Services\'].max()\n\nprint(f"The highest value of \'Cost of Goods and Services\' is: {highest_cost}")'}
== tool results: [{'python_answer': "KeyError('Cost of Goods and Services')"}]
I encountered an error. I will now view the data to understand the error.

running 2th step.
= running tool run_python_code, with parameters: {'code': "import pandas as pd\n\ndf = pd.read_csv('income_statement.csv')\n\nprint(df.columns)"}
== tool results: [{'python_answer': "Index(['Unnamed: 0', 'index',\n       'RevenueFromContractWithCustomerExcludingAssessedTax',\n       'CostOfGoodsAndServicesSold', 'GrossProfit',\n       'ResearchAndDevelopmentExpense',\n       'Selling

<a id="add_viewing_tool"></a>

### Add Viewing Tool 

What if we directly give the model the ability to view the data as a tool so that it can explicitly use it instead of indirectly figuring it out? 

In [27]:
def view_csv_data(path: str) -> dict:
    """
    Function to view the head, tail and shape of a given csv file.
    """
    df = pd.read_csv(path)

    return {
        "head": df.head().to_string(),
        "tail": df.tail().to_string(),
        "shape": str(df.shape),
    }

functions_map = {
    "run_python_code": run_python_code,
    "view_csv_data": view_csv_data
}

tools = [
    {
        "name": "run_python_code",
        "description": "given a python code, runs it",
        "parameter_definitions": {
            "code": {
                "description": "executable python code",
                "type": "str",
                "required": True
            }
        }
    },
    {
        "name": "view_csv_data",
        "description": "give path to csv data and get head, tail and shape of the data",
        "parameter_definitions": {
            "path": {
                "description": "path to csv",
                "type": "str",
                "required": True
            }
        }
    },
]

In [28]:
preamble = """
You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is `income_statement.csv`.
Always view the data first to write flawless code.
"""

question1 = "what is the highest value of cost of goods and service?"

output = cohere_agent(question1, preamble, tools, verbose=True)


running 0th step.
I will first view the data to understand the structure of the dataframe. Then, I will write and execute Python code to find the highest value of cost of goods and service.

running 1th step.
= running tool view_csv_data, with parameters: {'path': 'income_statement.csv'}
== tool results: [{'head': '   Unnamed: 0                  index  RevenueFromContractWithCustomerExcludingAssessedTax  CostOfGoodsAndServicesSold   GrossProfit  ResearchAndDevelopmentExpense  SellingGeneralAndAdministrativeExpense  OperatingExpenses  OperatingIncomeLoss  NonoperatingIncomeExpense  IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest  IncomeTaxExpenseBenefit  NetIncomeLoss  EarningsPerShareBasic  EarningsPerShareDiluted  WeightedAverageNumberOfSharesOutstandingBasic  WeightedAverageNumberOfDilutedSharesOutstanding\n0           0  2017-10-01-2018-09-29                                         265595000000                1.637560e+11  101839000000   

In [34]:
preamble = """
You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is `income_statement.csv`.
Always view the data first to write flawless code.
"""

question_dict = {
    'q1': ['what is the average value of each of column that is numeric in value, please add explanations too ?'],
    'q2': ['Analyze the data and give reasoning wether this data is good for analysis or not?'],
    'q3': ['Given the data what are the possible questions that can be asked from this data?'],
}


for qsn,val in question_dict.items():
    print(f'question:{qsn}')
    question = val[0]
    output = cohere_agent(question, preamble, tools, verbose=True)
    print('-'*50)

question:q1

running 0th step.
I will first view the data to understand its structure and then write and execute Python code to answer the user's request.

running 1th step.
= running tool view_csv_data, with parameters: {'path': 'income_statement.csv'}
== tool results: [{'head': '   Unnamed: 0                  index  RevenueFromContractWithCustomerExcludingAssessedTax  CostOfGoodsAndServicesSold   GrossProfit  ResearchAndDevelopmentExpense  SellingGeneralAndAdministrativeExpense  OperatingExpenses  OperatingIncomeLoss  NonoperatingIncomeExpense  IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest  IncomeTaxExpenseBenefit  NetIncomeLoss  EarningsPerShareBasic  EarningsPerShareDiluted  WeightedAverageNumberOfSharesOutstandingBasic  WeightedAverageNumberOfDilutedSharesOutstanding\n0           0  2017-10-01-2018-09-29                                         265595000000                1.637560e+11  101839000000                   1.423600e+10        

## CrewAI

Now let's Attemp to replicate the same thing using CrewAI but with GPT3.5

I liked using CrewAI more. It feels simpler and better explained. Their docs could use an update, but the idea behind it is nice.
The documentation recommends splitting the tasks into smaller chunks and assigning one agent for each task, but here, I just did one big chunk, and it works (kind of). It did work better on a different case when I split the task into smaller parts. I will make sure to post that later when it’s complete, and I’m happy with the results. 

In [16]:
from crewai import Agent, Task, Crew, Process

# Creating the task to combine results into a JSON format
python_evaluator = Task(
    description=(
        f"You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is f`{income_statement.head(3)}`. "
        "Always view the data first to write flawless code."
    ),
    expected_output="results of the code's output in a nice format",
    agent=Agent(
        role='Senior Python Engineer',
        goal='Successfully run the script and return the results in a useful format',
        verbose=True,
        memory=True,
        backstory=(
            "As an expert in data aggregation, your mission is to take the inputs and run the code to get the results. "
        ),
        allow_delegation=False,
        allow_code_execution=True,
    ),
)
crew = Crew(
        agents=[python_evaluator.agent],
        tasks=[python_evaluator],
        process=Process.sequential
    )

# Kickoff the crew with the provided text response
inputs = {'question_1': question_dict['q1'][0]}
result = crew.kickoff(inputs=inputs)

2024-06-30 19:11:07,534 - 8678138432 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer. To perform data aggregation and return the results in a useful format, I need to write a Python script using the pandas library. The goal is to take the provided dataframe and aggregate the data to get meaningful insights.

Here’s the code to aggregate the data and output the results:

```python
import pandas as pd
import numpy as np

# Sample data creation based on the provided dataframe structure
data = {
    "index": ["2017-10-01-2018-09-29", "2018-09-30-2018-12-29", "2018-09-30-2019-09-28"],
    "RevenueFromContractWithCustomerExcludingAssessedTax": [265595000000, 84310000000, 260174000000],
    "CostOfGoodsAndServicesSold": [163756000000, np.nan, 161782000000],
    "GrossProfit": [101839000000, 32031000000, 98392000000],
    "ResearchAndDevelopmentExpense": [14236000000, np.nan, 16217000000],
    "SellingGeneralAndAdministrativeExpense": [16705000000, np.nan, 18245000000],
    "OperatingExpenses": 

In [17]:
# Kickoff the crew with the provided text response
inputs = {'question_1': question_dict['q2'][0]}
result = crew.kickoff(inputs=inputs)



> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer. 

To effectively work with the given dataframe, I will first inspect the data to understand its structure. Then, I will perform data aggregation and analysis as required. Let's start by viewing the data.

```python
import pandas as pd

# Assuming the dataframe is already loaded and named `df`
print(df)
```

Once we have a clear view of the data, we can proceed with any specific calculations or transformations required.

Final Answer: Here is the complete content of the dataframe:

```plaintext
                   index  RevenueFromContractWithCustomerExcludingAssessedTax  \
0  2017-10-01-2018-09-29                                       265595000000     
1  2018-09-30-2018-12-29                                        84310000000     
2  2018-09-30-2019-09-28                                       260174000000     

   CostOfGoodsAndServicesSold   GrossProfit  ResearchAndDevelopmentExpense  \
0             

In [18]:
# Kickoff the crew with the provided text response
inputs = {'question_1': question_dict['q3'][0]}
result = crew.kickoff(inputs=inputs)



> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer by viewing the data and writing code to handle it effectively.

Let's start by viewing the data to ensure we understand its structure and then proceed with writing the appropriate code.

Here is the data in a pandas DataFrame:

```python
import pandas as pd
import numpy as np

data = {
    'index': ['2017-10-01-2018-09-29', '2018-09-30-2018-12-29', '2018-09-30-2019-09-28'],
    'RevenueFromContractWithCustomerExcludingAssessedTax': [265595000000, 84310000000, 260174000000],
    'CostOfGoodsAndServicesSold': [1.637560e+11, np.nan, 1.617820e+11],
    'GrossProfit': [101839000000, 32031000000, 98392000000],
    'ResearchAndDevelopmentExpense': [1.423600e+10, np.nan, 1.621700e+10],
    'SellingGeneralAndAdministrativeExpense': [1.670500e+10, np.nan, 1.824500e+10],
    'OperatingExpenses': [3.094100e+10, np.nan, 3.446200e+10],
    'OperatingIncomeLoss': [7.089800e+10, np.nan, 6.393000e+10],
    'NonoperatingI